#### Padronização e merge dos dados

In [79]:
import pandas as pd
import requests
from io import BytesIO

# 1.Leitura dos arquivos do IBGE, no link do github
def leitura(url):
    response = requests.get(url)
    return pd.read_excel(BytesIO(response.content))
    
url_parana = 'https://raw.githubusercontent.com/joaoandrebs/DataViz/main/Tabela%203.16%20-%20Parana.xlsx'
df_parana = leitura(url_parana)

url_rs = 'https://raw.githubusercontent.com/joaoandrebs/DataViz/main/Tabela%203.21%20-%20Rio%20Grande%20do%20Sul.xlsx'
df_rs = leitura(url_rs)

url_sc = 'https://raw.githubusercontent.com/joaoandrebs/DataViz/main/Tabela%203.24%20-%20Santa%20Catarina.xlsx'
df_sc = leitura(url_sc)

# 2. Padronização dos dataframes dos estados
df_sc = df_sc.iloc[4:]
df_sc = df_sc.rename(columns={
    'Tabela 1612 - Área plantada, área colhida, quantidade produzida, rendimento médio e valor da produção das lavouras temporárias': 'Cidade',
    'Unnamed: 1': 'Area Plantada',
    'Unnamed: 2': 'Area Colhida',
    'Unnamed: 5': 'Valor da produção'
})
df_sc['Estado'] = 'SC'

df_parana = df_parana.iloc[4:]
df_parana = df_parana.rename(columns={
    'Tabela 1612 - Área plantada, área colhida, quantidade produzida, rendimento médio e valor da produção das lavouras temporárias': 'Cidade',
    'Unnamed: 1': 'Area Plantada',
    'Unnamed: 2': 'Area Colhida',
    'Unnamed: 5': 'Valor da produção'
})
df_parana['Estado'] = 'Parana'

df_rs = df_rs.iloc[4:]
df_rs = df_rs.rename(columns={
    'Tabela 1612 - Área plantada, área colhida, quantidade produzida, rendimento médio e valor da produção das lavouras temporárias': 'Cidade',
    'Unnamed: 1': 'Area Plantada',
    'Unnamed: 2': 'Area Colhida',
    'Unnamed: 5': 'Valor da produção'})
df_rs['Estado'] = 'RS'

cachoeira1 = df_rs.index[df_rs['Cidade'] == '    Cachoeira do Sul'][0] # Remoção do primeiro registro de Cachoeira do sul, pq é uma região e não a cidade.
df_rs = df_rs.drop(cachoeira1)

# Bloco de regex para arrumar o nome Sant' Ana do livramento, pois não estava dando certo com loc.
import re
def corrigir_nome(nome):
    if isinstance(nome, str):
        nome = nome.strip()
        nome = re.sub(r"[‘’´`]", "'", nome)
        if re.fullmatch(r"Sant'? *Ana do Livramento", nome, flags=re.IGNORECASE):
            return "Sant' Ana do Livramento"
    return nome

df_rs['Cidade'] = df_rs['Cidade'].apply(corrigir_nome)

# 3. Convertendo as colunas para numero
df_total = pd.concat([df_parana, df_rs, df_sc])
df_total['Area Plantada'] = pd.to_numeric(df_total['Area Plantada'], errors='coerce')
df_total['Area Colhida'] = pd.to_numeric(df_total['Area Colhida'], errors='coerce')
df_total['Valor da produção'] = pd.to_numeric(df_total['Valor da produção'], errors='coerce')

# 4.  Excluir registros onde a coluna Cidade, seja exatamente um dos 3 estados
df_total = df_total[~df_total['Cidade'].isin(['Paraná', 'Rio Grande do Sul', 'Santa Catarina'])]


# 5. Excluir as cidades que estão no ranking das top 20

def limpar_texto(texto):
    if isinstance(texto, str):
        # remoçao espaços extras entre palavras e nas pontas
        return re.sub(r'\s+', ' ', texto).strip()
    return texto

# 7. Aplicar limpeza na coluna Cidade
df_total['Cidade'] = df_total['Cidade'].apply(limpar_texto)

excluir = [
    'Noroeste Rio-grandense',
    'Norte Central Paranaense',
    'Oeste Paranaense',
    'Centro-Sul Paranaense',
    'Sudoeste Rio-grandense',
    'Centro Ocidental Paranaense',
    'Noroeste Paranaense',
    'Sudoeste Paranaense',
    'Norte Pioneiro Paranaense',
    'Sudeste Paranaense',
    'Sudeste Rio-grandense',
    'Oeste Catarinense',
    'Centro Oriental Paranaense',
    'Metropolitana de Porto Alegre',
    'Centro Oriental Rio-grandense',
    'Campanha Ocidental',
    'Centro Ocidental Rio-grandense',    
    'Nordeste Rio-grandense',
    'Norte Catarinense',
    'Serrana',
    'Sul Catarinense',
    'Oeste Catarinense',
    'Norte Central Paranaense',
    'Oeste Paranaense',
    'Centro-Sul Paranaense',
    'Centro Ocidental Paranaense']

df_total = df_total[~df_total['Cidade'].isin(excluir)]
df_total.to_csv('df_total.csv', index=False, encoding='utf-8')

In [93]:
import pandas as pd
import geopandas as gpd
import folium
from folium.features import GeoJsonTooltip
from branca.element import Template, MacroElement
import unicodedata
import numpy as np

#1. Geojson e csvs - leitura
df = pd.read_csv('df_total.csv')
geojson_url = 'https://raw.githubusercontent.com/tbrugz/geodata-br/master/geojson/geojs-100-mun.json'
gdf_municipios = gpd.read_file(geojson_url)

# 2. Padronizar nomes das cidades
def padronizar(text):
    if isinstance(text, str):
        text = unicodedata.normalize('NFKD', text).encode('ascii','ignore').decode()
        return text.upper().strip()
    return text

df['Cidade'] = df['Cidade'].apply(padronizar)
gdf_municipios['name'] = gdf_municipios['name'].apply(padronizar)

# 3. Garantir numérico ----- não precisa, apenas evitando bug
df['Valor da produção'] = pd.to_numeric(df['Valor da produção'], errors='coerce')

# 4. Merge
gdf_final = gdf_municipios.merge(df, left_on='name', right_on='Cidade', how='left')

# 5. Criar mapa
m = folium.Map(location=[-26, -51], tiles='cartodbpositron', zoom_start=6)

# 6. Escala de cores: YlGnBu (amarelo→verde→azul)
threshold_scale = list(np.percentile(gdf_final['Valor da produção'].dropna(), [0, 20, 40, 60, 80, 100]))

folium.Choropleth(
    geo_data=gdf_final.to_json(),
    name='choropleth',
    data=gdf_final,
    columns=['name', 'Valor da produção'],
    key_on='feature.properties.name',
    fill_color='YlGnBu',
    fill_opacity=0.8,
    line_opacity=0.3,
    nan_fill_color='lightgray',
    threshold_scale=threshold_scale,
    highlight=True
).add_to(m)

# 7. destaque com hover no município
estilo = lambda x: {"fillColor": "white", "color": "black", "fillOpacity": 0.0001, "weight": 0.0001}
estilo_destaque = lambda x: {"fillColor": "darkblue", "color": "black", "fillOpacity": 0.5, "weight": 1}

highlight = folium.features.GeoJson(
    data=gdf_final,
    style_function=estilo,
    highlight_function=estilo_destaque,
    name="Destaque"
)

tooltip = GeoJsonTooltip(
    fields=["name"],
    aliases=["Município:"],
    style=("background-color: white; color: black; font-family: arial; font-size: 16px; padding: 10px;")
)
highlight.add_child(tooltip)
m.add_child(highlight)

# 8. Top 5 municípios de cada estado ----- 
ranking_html = "<p><b>5 Municípios com maior valor de produção agrícola em 2023 (mil reais)</b></p>"

for estado in df['Estado'].unique():
    top5 = (
        df[df['Estado'] == estado]
        .groupby('Cidade')['Valor da produção']
        .sum()
        .sort_values(ascending=False)
        .head(5)
    )
    ranking_html += f"<b>{estado}:</b><br>"
    for cidade, valor in top5.items():
        ranking_html += f"{cidade}: {valor:,.0f}<br>"
    ranking_html += "<br>"

# Caixa lateral fixa (legenda estilo -- HTML com CSS)
legend_html = f'''
     <div style="
         position: fixed; 
         top: 100px; 
         right: 10px; 
         width: 300px; 
         height: auto; 
         background-color: white;
         opacity: 0.9;
         z-index:9999;
         font-size:14px;
         padding: 15px;
         border:2px solid grey;
         box-shadow:2px 2px 5px rgba(0,0,0,0.3);
         overflow-y:auto;
     ">
     {ranking_html}
     </div>
'''

m.get_root().html.add_child(folium.Element(legend_html))

# Controle de camadas
folium.LayerControl().add_to(m)

m.save('cloropletico.html')
print('ok')


ok
